In [ ]:
using Random, Statistics, Printf

function rosenbrock(x, y)
    return (1 - x)^2 + 100 * (y - x^2)^2
end

function initialize_particles(num_particles, bounds)
    particles = [[rand(bounds[1]:0.01:bounds[2]), rand(bounds[1]:0.01:bounds[2])] for _ in 1:num_particles]
    velocities = [[rand(bounds[1]:0.01:bounds[2]), rand(bounds[1]:0.01:bounds[2])] for _ in 1:num_particles]
    personal_best = deepcopy(particles)
    personal_best_values = [rosenbrock(p[1], p[2]) for p in particles]

    global_best_index = argmin(personal_best_values)
    global_best = personal_best[global_best_index]
    global_best_value = personal_best_values[global_best_index]

    return particles, velocities, personal_best, personal_best_values, global_best, global_best_value
end

function particle_swarm_optimization(num_particles, bounds, w, c1, c2, max_iters, teleport=false)
    particles, velocities, personal_best, personal_best_values, global_best, global_best_value = initialize_particles(num_particles, bounds)

    for iter in 1:max_iters
        for i in 1:num_particles
            velocities[i] .= w .* velocities[i] .+
                             c1 .* rand() .* (personal_best[i] .- particles[i]) .+
                             c2 .* rand() .* (global_best .- particles[i])

            particles[i] .= particles[i] .+ velocities[i]

            particles[i] .= clamp.(particles[i], bounds[1], bounds[2])

            value = rosenbrock(particles[i][1], particles[i][2])
            if value < personal_best_values[i]
                personal_best[i] = copy(particles[i])
                personal_best_values[i] = value
            end

            if value < global_best_value
                global_best = copy(particles[i])
                global_best_value = value
            end
        end

        if teleport && iter % 10 == 0
            for i in 1:num_particles
                if rand() < 0.1
                    particles[i] = [rand(bounds[1]:0.01:bounds[2]), rand(bounds[1]:0.01:bounds[2])]
                end
            end
        end
    end

    return global_best, global_best_value
end

function test_num_particles()
    for num_particles in [10, 20, 50, 100, 200]
        start_time = time()
        best_point, best_value = particle_swarm_optimization(num_particles, (-2.0, 2.0), 0.5, 1.5, 1.5, 100)
        elapsed = time() - start_time
        @printf("Частиц: %d | Время: %.5f сек | Минимум: %.5f | Точка: (%.5f, %.5f)\n",
                num_particles, elapsed, best_value, best_point[1], best_point[2])
    end
end

function test_global_component()
    for c2 in [0.5, 1.0, 1.5, 2.0, 2.5]
        start_time = time()
        best_point, best_value = particle_swarm_optimization(50, (-2.0, 2.0), 0.5, 1.5, c2, 100)
        elapsed = time() - start_time
        @printf("c2: %.1f | Время: %.5f сек | Минимум: %.5f | Точка: (%.5f, %.5f)\n",
                c2, elapsed, best_value, best_point[1], best_point[2])
    end
end

function compare_teleportation()
    start_time = time()
    best_point_classic, best_value_classic = particle_swarm_optimization(50, (-2.0, 2.0), 0.5, 1.5, 1.5, 100)
    elapsed_classic = time() - start_time

    start_time = time()
    best_point_teleport, best_value_teleport = particle_swarm_optimization(50, (-2.0, 2.0), 0.5, 1.5, 1.5, 100, true)
    elapsed_teleport = time() - start_time

    @printf("Классический: Время: %.5f сек | Минимум: %.5f | Точка: (%.5f, %.5f)\n",
            elapsed_classic, best_value_classic, best_point_classic[1], best_point_classic[2])
    @printf("Телепортация: Время: %.5f сек | Минимум: %.5f | Точка: (%.5f, %.5f)\n",
            elapsed_teleport, best_value_teleport, best_point_teleport[1], best_point_teleport[2])
end

test_num_particles()
test_global_component()
compare_teleportation()

Частиц: 10 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
Частиц: 20 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
Частиц: 50 | Время: 0.00100 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
Частиц: 100 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
Частиц: 200 | Время: 0.00100 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
c2: 0.5 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
c2: 1.0 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
c2: 1.5 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
c2: 2.0 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
c2: 2.5 | Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
Классический: Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
Телепортация: Время: 0.00000 сек | Минимум: 0.00000 | Точка: (1.00000, 1.00000)
